In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
from datetime import date

In [ ]:
df =pd.read_csv('/kaggle/input/retail-demand-forecasting-dataset/Retail_Dataset2.csv',engine='python')
df.head()

In [ ]:
print('shape of dataset',np.shape(df))
print ('Null to Dataset Ratio in Dates: ',df.isnull().sum()[3]/df.shape[0]*100)
df.isnull().sum()

In [ ]:
df['Order_Demand'] = df['Order_Demand'].astype('int64')

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['month'] = df['Date'].dt.month

In [ ]:
import calendar
df['month_name'] = df['month'].apply(lambda x: calendar.month_abbr[x])


In [ ]:
df.sample(5)

In [ ]:
col =df.columns
def cnt(colm):
    print('value count in column',colm ,'is',df[colm].value_counts())
print(cnt('Open'))
print(cnt('Promo'))
print(cnt('SchoolHoliday'))
print(cnt('StateHoliday'))

# **we can see there are three type values in State hoilyday so we have to scale it .**

In [ ]:
df_copy =df.copy()

In [ ]:

def barplt(col):
    fig = plt.figure(figsize = (10, 8))
    splot=sns.countplot(df_copy[col])
    for p in splot.patches:
        splot.annotate(format(p.get_height(), '.1f'), 
                       (p.get_x() + p.get_width() / 2., p.get_height()), 
                       ha = 'center', va = 'center', 
                       xytext = (0, 9), 
                       textcoords = 'offset points')
    plt.xlabel(col, size=14)
    
    plt.show()
    


In [ ]:
barplt('Open')


In [ ]:
barplt('Promo')

In [ ]:
barplt('SchoolHoliday')

In [ ]:


barplt('StateHoliday')

In [ ]:
df.dtypes

In [ ]:
d_max= df['Date'].max()
d_min =df['Date'].min()
print('Data collection started from ',d_min,' to  ', d_max )

In [ ]:
print('Type of product is ',df['Product_Category'].nunique())
df['Product_Category'].value_counts()

In [ ]:
df.describe()

In [ ]:
cor =df.corr()
plt.figure(figsize=(15,8))
mask = np.zeros_like(cor)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(cor,mask =mask,annot =True)
plt.title('Corelation function among all variables')

plt.show()

In [ ]:
plt.figure(figsize=(20,14))
sns.countplot(df['Product_Category'].sort_values(ascending = True))
plt.xticks(rotation =90)
plt.title('Count plot of different catogery product demand')
plt.show()

In [ ]:
time_df = df.groupby('Date')['Order_Demand'].sum().reset_index()

time_df = time_df.set_index('Date')
time_df.head()


In [ ]:
#Averages daily sales value for the week, and we are using the start of each month as the timestamp.
y = time_df.Order_Demand.resample('W').mean()
print(np.shape(y))
y.sample(5)

In [ ]:
y.plot(figsize=(12,5))
plt.show()


In [ ]:
df2 = df[['month', 'Warehouse', 'Order_Demand']].groupby(['month', 'Warehouse'], as_index=False).count()
df2  = df2.pivot(index='month', columns='Warehouse', values='Order_Demand')
df2.head()

In [ ]:
df_new = df[['month_name', 'Warehouse', 'Order_Demand']].groupby(['month_name', 'Warehouse'], as_index=False).count()
df_new  = df_new.pivot(index='month_name', columns='Warehouse', values='Order_Demand')
df_new.head()

In [ ]:
df2.index = df2.index.map(int)
df2.plot(kind='area', figsize=(20, 10))

plt.title('Order_Demand Trend')
plt.ylabel('Number of Order_Demand')
plt.xlabel('Month')
plt.show()

In [ ]:
df2[['Whse_A', 'Whse_C', 'Whse_J', 'Whse_S']].plot(kind='box', figsize=(8, 6))

plt.title('Order_Demand Trend [2011 - 2017]')
plt.ylabel('Number of Order_Demand')

plt.show()

In [ ]:
df2.index = df2.index.map(int)
df2.plot(kind='barh', figsize=(20, 10))

plt.title('Order_Demand Trend')
plt.xlabel('Number of Order_Demand')

plt.show()

In [ ]:
df2[['Whse_A', 'Whse_C', 'Whse_J', 'Whse_S']].plot(kind='line', figsize=(10, 8))

plt.title('Order_Demand Trend [2011 - 2017]')
plt.ylabel('Number of Order_Demand')

plt.show()

In [ ]:
df_new['Total'] = df_new.sum(axis=1)

In [ ]:
df_new.head()

In [ ]:
#df2.index = df2.index.map(int)
df_new.plot.pie(y='Total', figsize=(20, 10),
                            autopct='%1.1f%%',               
                            )

plt.title('Order_Demand Trend')
plt.legend(labels=df_new.index)
plt.show()

In [ ]:
df.head()

In [ ]:
df_copy =df.copy()

df_copy['StateHoliday'].replace('a',1,inplace=True)
df_copy['StateHoliday'].replace('b',2,inplace=True)

df_copy.head()

In [ ]:
df_copy.columns

In [ ]:
df_f =df_copy.drop(['Warehouse','month', 'Date','Product_Code','Product_id','Product_Category','month_name'],axis=1)

df_f[['Promo']] = df_f[['Promo']].astype(int)
df_f[['Open']] = df_f[['Open']].astype(int)
df_f[['SchoolHoliday']] = df_f[['SchoolHoliday']].astype(int)
df_f[['StateHoliday']] = df_f[['StateHoliday']].astype(int)
print(df_f.dtypes)
df_f.head(7)


In [ ]:
df_f.to_csv('DF_F.csv')

In [ ]:

from IPython.display  import Image
import os
Image('../input/image12/insight1.png')
     

In [ ]:
df.columns

In [ ]:
df['Product_Category']=df['Product_Category'].sort_values(ascending=False)

In [ ]:
df_cat = df[['month', 'Product_Category', 'Order_Demand']].groupby(['month', 'Product_Category'], as_index=False).count()
df_cat  = df_cat.pivot(index='month', columns='Product_Category', values='Order_Demand')
print(np.shape(df_cat))
df_cat.head()

In [ ]:
df['Product_Category'].value_counts()

In [ ]:
df_cat = df_cat[['Category_019','Category_005','Category_001','Category_007','Category_021','Category_006']] 

In [ ]:
print(np.shape(df_cat))
df_cat.head()


In [ ]:
df_cat.plot(figsize=(12,5))
plt.show()